In [2]:
import boto3

In [3]:
import pandas as pd
from io import StringIO

In [4]:
# Set up S3 client
client = boto3.client('s3')

In [5]:
folders_list =  ['level_country_10_09_2023']

In [6]:
# Bucket name and common prefix
bucket_forecast = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
common_path = 'Forecast/Forecast_analysis/'

<a data-toggle="collapse" href="#collapseExample" role="button" aria-expanded="false" aria-controls="collapseExample">
  <span style="color:green; font-size:2em; font-weight:bold;">Generating DF</span>
</a>

In [7]:
# Initialize a list to store DataFrames
dfs = []

In [8]:
# Iterate over each folder in the list
for folder_item in folders_list:
    # Build the new path for the current folder
    path = f'{common_path}{folder_item}/'
    
    # List objects in the folder
    elements = client.list_objects(Bucket=bucket_forecast, Prefix=path)

    # Iterate over each object in the folder
    for obj in elements.get('Contents', []):
        # Check if the object is a CSV file
        if obj['Key'].endswith('.csv'):
            # Read CSV content from S3
            response = client.get_object(Bucket=bucket_forecast, Key=obj['Key'])
            csv_content = response['Body'].read().decode('utf-8')

            # Transform CSV content to DataFrame
            df = pd.read_csv(StringIO(csv_content))

            
            df['folder_id'] = folder_item # 'folder_id' column to indentify the forecast

            dfs.append(df)

In [9]:
# Concatenate all DataFrames into a single one
final_df = pd.concat(dfs, ignore_index=True)

In [10]:
final_df['folder_id'].unique()

array(['level_country_10_09_2023'], dtype=object)

In [11]:
final_df['date'] = pd.to_datetime(final_df['date']).dt.date

In [12]:
final_df.rename(columns={'item_id':'country'}, inplace=True)

In [13]:
final_df = final_df.applymap(lambda x: x.upper() if isinstance(x, str) else x)

/tmp/ipykernel_34/2399063668.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  final_df = final_df.applymap(lambda x: x.upper() if isinstance(x, str) else x)


In [14]:
#final_df = final_df[['folder_id', 'date', 'country','p10', 'p50', 'p90', 'mean']]
final_df = final_df[final_df['country'] == 'MEXICO'][['folder_id', 'date', 'country', 'p10', 'p50', 'p90', 'mean']]

In [15]:
final_df['date'].nunique()

7

<div class="collapse" id="collapseExample">

In [16]:
final_df

,folder_id,date,country,p10,p50,p90,mean
80,LEVEL_COUNTRY_10_09_2023,2023-09-11,MEXICO,1.820240e+07,2.140862e+07,2.461483e+07,2.140862e+07
81,LEVEL_COUNTRY_10_09_2023,2023-09-12,MEXICO,1.465310e+07,1.733028e+07,2.000746e+07,1.733028e+07
218,LEVEL_COUNTRY_10_09_2023,2023-09-11,MEXICO,1.797258e+07,1.985797e+07,2.144531e+07,1.973429e+07
219,LEVEL_COUNTRY_10_09_2023,2023-09-12,MEXICO,1.477272e+07,1.613229e+07,1.762952e+07,1.608681e+07
220,LEVEL_COUNTRY_10_09_2023,2023-09-13,MEXICO,1.383615e+07,1.499070e+07,1.662120e+07,1.509782e+07
221,LEVEL_COUNTRY_10_09_2023,2023-09-14,MEXICO,1.563761e+07,1.686265e+07,1.872429e+07,1.694243e+07
222,LEVEL_COUNTRY_10_09_2023,2023-09-15,MEXICO,2.354739e+07,2.532426e+07,2.743517e+07,2.533673e+07
223,LEVEL_COUNTRY_10_09_2023,2023-09-16,MEXICO,2.519971e+07,2.727892e+07,2.966322e+07,2.741284e+07
224,LEVEL_COUNTRY_10_09_2023,2023-09-17,MEXICO,2.396527e+07,2.615927e+07,2.861282e+07,2.629443e+07


In [17]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'daily_check'

In [21]:
#pip install pyathena

In [22]:
from pyathena import connect

#Connection
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')
df_daily_check = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)

/tmp/ipykernel_34/268183201.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_daily_check = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)


In [23]:
df_daily_check['date'] = pd.to_datetime(df_daily_check['date']).dt.date

In [24]:
df_daily_check.head()

,payer,country,date,tx,amount,coupon_count
0,TELECOMM TELEGRAFOS (APPRIZA),MEXICO,2023-04-24,977,536661.67,22
1,ELEKTRA (BTS),GUATEMALA,2021-02-27,157,24916.09,121
2,OXXO (TN),MEXICO,2021-10-05,210,15696.39,37
3,TRANSFER DIRECTO,MEXICO,2023-08-12,123,80301.68,2
4,BANCO DE GUAYAQUIL,ECUADOR,2021-12-31,58,15688.22,2


In [25]:
grouped_daily = df_daily_check.groupby(['date','country']).agg({
    'amount': 'sum',
    'tx': 'sum',
    'coupon_count': 'sum'
}).reset_index()

In [26]:
grouped_daily['amount'].sum()

25110624454.414898

In [27]:
# Merge both df, keeping only daily_check data related to forecast
df_forecast = pd.merge(grouped_daily, final_df, on=['date','country'], how='inner').drop_duplicates()

In [28]:
df_forecast['date'].nunique()

7

In [29]:
df_forecast

,date,country,amount,tx,coupon_count,folder_id,p10,p50,p90,mean
0,2023-09-11,MEXICO,17073946.44,34597,593,LEVEL_COUNTRY_10_09_2023,1.820240e+07,2.140862e+07,2.461483e+07,2.140862e+07
1,2023-09-11,MEXICO,17073946.44,34597,593,LEVEL_COUNTRY_10_09_2023,1.797258e+07,1.985797e+07,2.144531e+07,1.973429e+07
2,2023-09-12,MEXICO,13249850.38,26801,424,LEVEL_COUNTRY_10_09_2023,1.465310e+07,1.733028e+07,2.000746e+07,1.733028e+07
3,2023-09-12,MEXICO,13249850.38,26801,424,LEVEL_COUNTRY_10_09_2023,1.477272e+07,1.613229e+07,1.762952e+07,1.608681e+07
4,2023-09-13,MEXICO,12225455.26,24402,397,LEVEL_COUNTRY_10_09_2023,1.383615e+07,1.499070e+07,1.662120e+07,1.509782e+07
5,2023-09-14,MEXICO,13377976.16,26891,474,LEVEL_COUNTRY_10_09_2023,1.563761e+07,1.686265e+07,1.872429e+07,1.694243e+07
6,2023-09-15,MEXICO,21357505.72,46491,891,LEVEL_COUNTRY_10_09_2023,2.354739e+07,2.532426e+07,2.743517e+07,2.533673e+07
7,2023-09-16,MEXICO,23379643.53,52205,976,LEVEL_COUNTRY_10_09_2023,2.519971e+07,2.727892e+07,2.966322e+07,2.741284e+07
8,2023-09-17,MEXICO,24184710.24,52207,850,LEVEL_COUNTRY_10_09_2023,2.396527e+07,2.615927e+07,2.861282e+07,2.629443e+07


In [30]:
df_forecast["pe_mean"] = abs((df_forecast['amount'] - df_forecast['mean']) / df_forecast['amount'])
df_forecast["pe_P10"] = abs((df_forecast['amount'] - df_forecast['p10']) / df_forecast['amount'])
df_forecast["pe_P50"] = abs((df_forecast['amount'] - df_forecast['p50']) / df_forecast['amount'])

In [31]:
df_forecast.pe_mean.min(), df_forecast.pe_mean.max()

(0.08723378279350441, 0.307960271392282)

In [32]:
df_forecast

,date,country,amount,tx,coupon_count,folder_id,p10,p50,p90,mean,pe_mean,pe_P10,pe_P50
0,2023-09-11,MEXICO,17073946.44,34597,593,LEVEL_COUNTRY_10_09_2023,1.820240e+07,2.140862e+07,2.461483e+07,2.140862e+07,0.253876,0.066092,0.253876
1,2023-09-11,MEXICO,17073946.44,34597,593,LEVEL_COUNTRY_10_09_2023,1.797258e+07,1.985797e+07,2.144531e+07,1.973429e+07,0.155813,0.052632,0.163057
2,2023-09-12,MEXICO,13249850.38,26801,424,LEVEL_COUNTRY_10_09_2023,1.465310e+07,1.733028e+07,2.000746e+07,1.733028e+07,0.307960,0.105907,0.307960
3,2023-09-12,MEXICO,13249850.38,26801,424,LEVEL_COUNTRY_10_09_2023,1.477272e+07,1.613229e+07,1.762952e+07,1.608681e+07,0.214112,0.114935,0.217545
4,2023-09-13,MEXICO,12225455.26,24402,397,LEVEL_COUNTRY_10_09_2023,1.383615e+07,1.499070e+07,1.662120e+07,1.509782e+07,0.234950,0.131750,0.226187
5,2023-09-14,MEXICO,13377976.16,26891,474,LEVEL_COUNTRY_10_09_2023,1.563761e+07,1.686265e+07,1.872429e+07,1.694243e+07,0.266442,0.168907,0.260478
6,2023-09-15,MEXICO,21357505.72,46491,891,LEVEL_COUNTRY_10_09_2023,2.354739e+07,2.532426e+07,2.743517e+07,2.533673e+07,0.186315,0.102534,0.185731
7,2023-09-16,MEXICO,23379643.53,52205,976,LEVEL_COUNTRY_10_09_2023,2.519971e+07,2.727892e+07,2.966322e+07,2.741284e+07,0.172509,0.077848,0.166781
8,2023-09-17,MEXICO,24184710.24,52207,850,LEVEL_COUNTRY_10_09_2023,2.396527e+07,2.615927e+07,2.861282e+07,2.629443e+07,0.087234,0.009074,0.081645


In [33]:
# OVERALL ACCURACY
df_forecast['pe_mean'].mean(), df_forecast['pe_P10'].mean(), df_forecast['pe_P50'].mean()

(0.20880135162364455, 0.09218653358508652, 0.2070289921933232)

In [34]:
# Forecast Precision
(df_forecast['p90'] - df_forecast['p10']).mean()

4107432.8427375373

In [35]:
# Precision
specific_date_country = df_forecast.loc[0]
specific_date_country_accuracy_mean = specific_date_country['pe_mean'].mean()
specific_date_country_accuracy_P10 = specific_date_country['pe_P10'].mean()
specific_date_country_accuracy_P50 = specific_date_country['pe_P50'].mean()
specific_date_country_precision = (specific_date_country['p90'] - specific_date_country['p10']).mean()
specific_date_country_precision

6412429.7432798445

In [36]:
import sagemaker
from sagemaker import get_execution_role

#Set up SageMaker session and role
sagemaker_session = sagemaker.Session()
role = 'arn:aws:iam::283731589572:role/service-role/AmazonSageMaker-ExecutionRole-20231127T122316'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [40]:
#pip install awswrangler

In [41]:
#pip install awswrangler[redshift]

In [42]:
import awswrangler as wr

database = 'viamericas'
cluster_id = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
sql_table = 'forecast_mexico'
redshift_role_arn = 'arn:aws:iam::283731589572:role/redshift-role'

# Connect to Redshift with IAM authentication
wr.redshift.connect(database=database, cluster_identifier=cluster_id, iam_role=redshift_role_arn, dbname=database)


InvalidArgumentCombination: Failed attempt to connect. You MUST pass a connection name (Glue Catalog) OR a secret_id as argument.

In [42]:
# Creating the table @ RS (replace if exists)

wr.redshift.to_sql(
    dataframe=df_forecast,
    table=sql_table,
    schema=database,
    index=False,
    database=database,
    cluster_identifier=cluster_id,
    iam_role=redshift_role_arn,
    mode='overwrite',  # Overwrite mode is used for both creating and replacing
    if_exists='replace'  # Replace the table if it exists, create otherwise
)

TypeError: got an unexpected keyword argument 'dataframe'

In [ ]:
# Close the Redshift connection
wr.redshift.close()